In [6]:
import math
import torch
from torch import nn
from models.lxrt_adaptive import VQAModel_Adaptive

In [2]:
params = {'adapt_span_enabled': True, 'attn_span': 1024, 'adapt_span_loss_coeff': 0.000005, 
          'adapt_span_ramp': 32, 'adapt_span_init': 0.002, 'adapt_span_cache': True, 'nb_heads': 12,
          'bs': 256, 'mask_size': [20,36], 'sparse_enabled': True, 'num_attention_heads': 4,
          'layer_sizes': {'lang':6,'cross':4,'vision':4}, 'from_scratch': False }

In [3]:
model = VQAModel_Adaptive(3129,params)

I0203 20:25:08.482328 140630299241344 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/u37216/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0203 20:25:08.920816 140630299241344 lxmert_utils.py:200] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/u37216/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
I0203 20:25:08.927352 140630299241344 lxmert_utils.py:208] extracting archive file /home/u37216/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /home/u37216/tmp/tmpg_oh5t3j
I0203 20:25:16.659927 140630

LXRT encoder with 6 l_layers, 4 x_layers, and 4 r_layers.
Using Adaptive Variant
Sparse Enabled


In [4]:
lang_feats = torch.rand(256,20,768)
lang_attention_mask = torch.rand(256,1,1,20)
visn_feats = torch.rand(256,36,768)
visn_attention_mask = None

In [5]:
layer_ml[0](lang_feats,lang_attention_mask).shape

In [16]:
class LayerDrop_Bert(nn.Module):
    def __init__(self, module_list, drop_pct=0.2):
        super().__init__()
        self.module_list = module_list
        self.drop_pct = int(drop_pct*10)
        self.length = len(module_list)
    
    def forward(self, feats, attention_mask):
        x = torch.randint(0, self.length, (self.drop_pct,))
        print(x)
        for index, layer in enumerate(self.module_list):
            if index not in x:
                feats = layer(feats, attention_mask) 
                print(index)
        return feats
    
class LayerDrop_Cross(nn.Module):
    def __init__(self, module_list, drop_pct=0.2):
        super().__init__()
        self.module_list = module_list
        self.drop_pct = int(drop_pct*10)
        self.length = len(module_list)
    
    def forward(self, lang_feats, lang_attention_mask, visn_feats, visn_attention_mask):
        x = torch.randint(0, self.length, (self.drop_pct,)) 
        print(x)
        for index, layer in enumerate(self.module_list):
            if index not in x:
                lang_feats, visn_feats = layer(lang_feats, lang_attention_mask,
                                               visn_feats, visn_attention_mask) # 
                print(index)
        return lang_feats, visn_feats

torch.Size([256, 20, 768])


In [ ]:
l = LayerDrop_Bert(layer_ml,0.2)

In [ ]:
l(lang_feats, lang_attention_mask).shape

In [ ]:
cross_ml = model.lxrt_encoder.model.bert.encoder.x_layers

In [ ]:
l = LayerDrop_Cross(cross_ml,0.2)

In [ ]:
l(lang_feats=lang_feats, lang_attention_mask=lang_attention_mask,
                      visn_feats=visn_feats, visn_attention_mask=visn_attention_mask)[0].shape

In [ ]:
len(cross_ml)